In [ ]:
import os
import random
import argparse
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

import torch
import torch.nn as nn
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# from config_run import *
# from config_debug import *
# from models.AIOM import AIOM
# from load_data import MMDataLoader

In [ ]:
seed = 1
args = {'modelName': 'tfn', 'datasetName': 'msazh', 'num_workers': 0, 'modality': 'M'}
save_path = 'asserts' + '/' + args['modelName'] + '-%d' % seed + '.pdf'
config = Config(args)
args = config.get_config()
device = torch.device('cuda: 1')
# load model
model_save_pathes = glob(os.path.join('/home/iyuge2/Project/MSA-ZH/models_seeds', '%d-' % seed + args.modelName + '-msazh-' + args.modality + '-*.pth'))
model = Terminator(args)
assert len(model_save_pathes) == 1
model.load_state_dict(torch.load(model_save_pathes[0]))
model.to(device)
model.eval()
# load data
dataloader = MMDataLoader(args)

In [ ]:
feature_ts, feature_as, feature_vs, feature_fs = [], [], [], []
y_pred = []
y_true = []
with torch.no_grad():
    for mode in ['test']:
        with tqdm(dataloader[mode]) as td:
            for batch_data in td:
                vision = batch_data['vision'].to(device)
                audio = batch_data['audio'].to(device)
                text = batch_data['text'].to(device)
                labels_M = batch_data['labels_M'].to(device).view(-1, 1)
                labels_T = batch_data['labels_T'].to(device).view(-1, 1)
                labels_A = batch_data['labels_A'].to(device).view(-1, 1)
                labels_V = batch_data['labels_V'].to(device).view(-1, 1)

                outputs = model(text, audio, vision)
                # save features and results
                feature_ts.append(outputs['Feature_t'].cpu().detach())
                feature_as.append(outputs['Feature_a'].cpu().detach())
                feature_vs.append(outputs['Feature_v'].cpu().detach())
                feature_fs.append(outputs['Feature_f'].cpu().detach())
                y_pred.append(outputs['M'].cpu())
                y_true.append(eval('labels_M').cpu())

In [ ]:
feature_T = torch.cat(feature_ts, dim=0).cpu().detach().numpy()
feature_A = torch.cat(feature_as, dim=0).cpu().detach().numpy()
feature_V = torch.cat(feature_vs, dim=0).cpu().detach().numpy()
feature_F = torch.cat(feature_fs, dim=0).cpu().detach().numpy()
y_pred = torch.cat(y_pred, dim=0).view(-1).cpu().detach().numpy()
y_pred[y_pred > 0] = 1
y_pred[y_pred <= 0] = 0

y_true = torch.cat(y_true, dim=0).view(-1).cpu().detach().numpy()
y_true[y_true > 0] = 1
y_true[y_true <= 0] = 0
sum(y_pred == 1)

In [ ]:
print(feature_T.shape)
print(feature_A.shape)
print(feature_V.shape)

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2, whiten=True)

result_T = pca.fit_transform(feature_T)
result_A = pca.fit_transform(feature_A)
result_V = pca.fit_transform(feature_V)

result_T = result_T.reshape(1, -1)
result_A = result_A.reshape(1, -1)
result_V = result_V.reshape(1, -1)
results = np.concatenate([result_T, result_A, result_V], axis=0)

In [ ]:
fdiff = np.corrcoef(results)[:3, :3]
fdiff

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
plt.rcParams['font.sans-serif'] = ['Times New Roman']
tsne = TSNE(n_components=2, init='pca', random_state=0)
result_T = tsne.fit_transform(feature_T)
result_A = tsne.fit_transform(feature_A)
result_V = tsne.fit_transform(feature_V)
att_xs = [result_T, result_A, result_V]
colors = ['r', 'g', 'b']
labels = ['T', 'A', 'V']
for i in range(len(colors)): 
    plt.scatter(att_xs[i][:,0:1], att_xs[i][:,1:2], label=labels[i], c = colors[i])
    plt.legend(loc='upper right')
plt.savefig(save_path, dpi=600, format='pdf')